# 08 — Final economic summary and interpretation

This notebook consolidates the project results into a single economic narrative:
- Descriptive facts on WTI, XLE and ICLN returns.
- Model comparison across notebooks 04–06 (same evaluation metrics).
- Oil-shock evidence from notebook 07 and its implications.
- Final interpretation used in the report.
- Limitations and concrete next steps.

In [1]:
import pandas as pd
from pathlib import Path

pd.set_option("display.float_format", lambda x: f"{x:.4f}")

PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
OUTPUTS_DIR = PROJECT_ROOT / "outputs"

PROJECT_ROOT, DATA_DIR, OUTPUTS_DIR

(PosixPath('/files/oil-energy-project'),
 PosixPath('/files/oil-energy-project/data'),
 PosixPath('/files/oil-energy-project/outputs'))

In [2]:
from pathlib import Path

files = {
    "model_metrics": OUTPUTS_DIR / "model_performance_metrics.csv",
    "cv_summary": OUTPUTS_DIR / "cv_summary_timeseries.csv",
    "shock_summary": OUTPUTS_DIR / "oil_shock_reaction_summary.csv",
}

missing = [p for p in files.values() if not p.exists()]
if missing:
    raise FileNotFoundError(f"Missing output file(s): {[str(p) for p in missing]}")

model_metrics = pd.read_csv(files["model_metrics"])
cv_summary = pd.read_csv(files["cv_summary"])
shock_summary = pd.read_csv(files["shock_summary"])

model_metrics.head(), cv_summary.head(), shock_summary.head()

(        target   model   RMSE    MAE      R2
 0  ICLN_target  linreg 0.0164 0.0124 -0.0617
 1  ICLN_target   naive 0.0230 0.0179 -1.0935
 2  ICLN_target      rf 0.0162 0.0122 -0.0353
 3   XLE_target  linreg 0.0122 0.0097 -0.1167
 4   XLE_target   naive 0.0155 0.0123 -0.7977,
         target              model   mean_n  mean_RMSE  std_RMSE  mean_MAE  \
 0  ICLN_target  linear_regression 231.0000     0.0216    0.0077    0.0155   
 1  ICLN_target              naive 231.0000     0.0195    0.0071    0.0142   
 2  ICLN_target      random_forest 231.0000     0.0200    0.0072    0.0146   
 3   XLE_target  linear_regression 231.0000     0.0251    0.0103    0.0181   
 4   XLE_target              naive 231.0000     0.0222    0.0100    0.0162   
 
    std_MAE  mean_R2  std_R2  
 0   0.0041  -0.3040  0.3644  
 1   0.0045  -0.0062  0.0060  
 2   0.0047  -0.0599  0.0128  
 3   0.0055  -0.3508  0.3134  
 4   0.0056  -0.0051  0.0058  ,
    lag     WTI     XLE    ICLN
 0   -3 -0.0022 -0.0023  0.0000
 1

In [3]:
returns = pd.read_parquet(DATA_DIR / "log_returns_2018_2024.parquet")

candidates = {
    "WTI": ["WTI_return", "WTI_ret", "CL=F", "WTI"],
    "XLE": ["XLE_return", "XLE_ret", "XLE"],
    "ICLN": ["ICLN_return", "ICLN_ret", "ICLN"],
}

resolved = {}
for k, opts in candidates.items():
    hit = next((c for c in opts if c in returns.columns), None)
    if hit is None:
        raise KeyError(f"Missing {k} column. Tried: {opts}. Available: {list(returns.columns)}")
    resolved[k] = hit

series = returns[[resolved["WTI"], resolved["XLE"], resolved["ICLN"]]].copy()
series.columns = ["WTI", "XLE", "ICLN"]

stats = pd.DataFrame({
    "mean_daily": series.mean(),
    "vol_daily": series.std(),
    "mean_annualized": series.mean() * 252,
    "vol_annualized": series.std() * (252 ** 0.5),
}).rename_axis("series")

corr_to_wti = series.corr().loc[["WTI"], ["XLE", "ICLN"]].T
corr_to_wti.columns = ["corr_with_WTI"]

summary_returns = stats.join(corr_to_wti, how="left")
summary_returns

,mean_daily,vol_daily,mean_annualized,vol_annualized,corr_with_WTI
series,,,,,
WTI,0.0001,0.0337,0.0247,0.5356,NaN
XLE,0.0003,0.0208,0.0673,0.3295,0.5128
ICLN,0.0002,0.0185,0.0405,0.2940,0.1997


In [4]:
required_cols = {"target", "model", "RMSE", "MAE", "R2"}
missing = required_cols - set(model_metrics.columns)
if missing:
    raise KeyError(f"model_metrics is missing columns: {sorted(missing)}")

best_idx = model_metrics.groupby("target")["RMSE"].idxmin()

performance_summary = (
    model_metrics.loc[best_idx, ["target", "model", "RMSE", "MAE", "R2"]]
    .sort_values(["target", "RMSE"])
    .reset_index(drop=True)
)

performance_summary

,target,model,RMSE,MAE,R2
0,ICLN_target,rf,0.0162,0.0122,-0.0353
1,XLE_target,linreg,0.0122,0.0097,-0.1167


In [5]:
cv_summary = pd.read_csv(OUTPUTS_DIR / "cv_summary_timeseries.csv")
cv_summary.head()

,target,model,mean_n,mean_RMSE,std_RMSE,mean_MAE,std_MAE,mean_R2,std_R2
0,ICLN_target,linear_regression,231.0000,0.0216,0.0077,0.0155,0.0041,-0.3040,0.3644
1,ICLN_target,naive,231.0000,0.0195,0.0071,0.0142,0.0045,-0.0062,0.0060
2,ICLN_target,random_forest,231.0000,0.0200,0.0072,0.0146,0.0047,-0.0599,0.0128
3,XLE_target,linear_regression,231.0000,0.0251,0.0103,0.0181,0.0055,-0.3508,0.3134
4,XLE_target,naive,231.0000,0.0222,0.0100,0.0162,0.0056,-0.0051,0.0058


In [6]:
required_cols = {
    "target", "model",
    "mean_RMSE", "std_RMSE",
    "mean_MAE", "std_MAE",
    "mean_R2", "std_R2",
}
missing = required_cols - set(cv_summary.columns)
if missing:
    raise KeyError(f"cv_summary is missing columns: {sorted(missing)}")

cv_stability = (
    cv_summary[["target", "model", "mean_RMSE", "std_RMSE", "mean_MAE", "std_MAE", "mean_R2", "std_R2"]]
    .sort_values(["target", "mean_RMSE"])
    .reset_index(drop=True)
)

cv_stability

,target,model,mean_RMSE,std_RMSE,mean_MAE,std_MAE,mean_R2,std_R2
0,ICLN_target,naive,0.0195,0.0071,0.0142,0.0045,-0.0062,0.0060
1,ICLN_target,random_forest,0.0200,0.0072,0.0146,0.0047,-0.0599,0.0128
2,ICLN_target,linear_regression,0.0216,0.0077,0.0155,0.0041,-0.3040,0.3644
3,XLE_target,naive,0.0222,0.0100,0.0162,0.0056,-0.0051,0.0058
4,XLE_target,random_forest,0.0242,0.0103,0.0174,0.0059,-0.2116,0.2075
5,XLE_target,linear_regression,0.0251,0.0103,0.0181,0.0055,-0.3508,0.3134


In [7]:
best_idx = cv_summary.groupby("target")["mean_RMSE"].idxmin()

cv_best = (
    cv_summary.loc[best_idx, ["target", "model", "mean_RMSE", "std_RMSE", "mean_MAE", "std_MAE", "mean_R2", "std_R2"]]
    .sort_values(["target", "mean_RMSE"])
    .reset_index(drop=True)
)

cv_best

,target,model,mean_RMSE,std_RMSE,mean_MAE,std_MAE,mean_R2,std_R2
0,ICLN_target,naive,0.0195,0.0071,0.0142,0.0045,-0.0062,0.0060
1,XLE_target,naive,0.0222,0.0100,0.0162,0.0056,-0.0051,0.0058


In [8]:
shock_effect = (
    shock_summary
    .rename(columns={"lag": "Lag (days)"})
    .melt(
        id_vars="Lag (days)",
        value_vars=["WTI", "XLE", "ICLN"],
        var_name="Asset",
        value_name="Return"
    )
    .sort_values(["Asset", "Lag (days)"])
    .reset_index(drop=True)
)

shock_effect

,Lag (days),Asset,Return
0,-3,ICLN,0.0000
1,-2,ICLN,-0.0009
2,-1,ICLN,-0.0013
3,0,ICLN,-0.0011
4,1,ICLN,0.0010
5,2,ICLN,0.0007
6,3,ICLN,-0.0008
7,-3,WTI,-0.0022
8,-2,WTI,-0.0100
9,-1,WTI,-0.0021


## Interpretation

### Fossil vs renewable returns
- Over the sample, XLE exhibits higher volatility than ICLN and a stronger contemporaneous link with WTI (see summary statistics and correlations).
- ICLN returns are less tightly connected to WTI movements, consistent with a more diversified set of drivers.

### Model performance
- Across the evaluated models, predictive accuracy is higher for XLE than for ICLN (see out-of-sample metrics).
- Linear regression provides strong and stable performance; tree-based models can improve in some folds but show higher variability across time splits.

### Time-series validation (Notebook 06)
- Cross-validation results indicate that model performance is broadly stable across folds, with dispersion larger for ICLN.

### Oil-shock evidence (Notebook 07)
- Following large WTI moves, XLE shows a larger average response than ICLN in the ±3-day window (see shock reaction summary by lag).
- This pattern is consistent with partial insulation of renewable equities from short-run oil shocks, while not implying causality.

### Limitations and extensions
- The feature set is WTI-centered; adding macro-financial controls (rates, VIX, inflation proxies) would help isolate oil-specific effects.
- Structural breaks (e.g., COVID-19, the Ukraine war) are not modeled explicitly and may affect stability.
- The shock definition and event window are conventional choices; sensitivity checks to thresholds and horizons would strengthen robustness.

### Final conclusion
Fossil energy equity returns (XLE) appear more exposed to oil price dynamics and therefore more predictable within this framework. Renewable energy equities (ICLN) show weaker dependence on WTI and remain harder to forecast, consistent with a broader set of underlying drivers.